In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, SpatialDropout2D
from keras.layers import Input, merge, UpSampling2D, BatchNormalization
from keras.optimizers import Adam,rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import time
import gc
import os

import SimpleITK as sitk
import numpy as np
import csv
import scipy
from glob import glob
import pandas as pd
from scipy import ndimage
from tqdm import tqdm 
import pandas as pd

from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature,draw,data
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc
import numpy as np



import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from numba import autojit
import zarr
from PIL import Image
import cv2
import scipy.spatial.distance as dist
import gc
import warnings
from solo.unet_utils import *
from solo.unet_models import *
#from solo.preproc_utils import *
#from solo.tianchi import *

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras import backend as K
K.set_image_dim_ordering('th') 


#from inception_unet import *
np.random.seed(1337)



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


warnings.filterwarnings("ignore")

In [6]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/Volumes/solo/ali/pic/train/'
validation_data_dir = '/Volumes/solo/ali/pic/val/'
nb_train_samples = 1800
nb_validation_samples = 500
epochs = 10
batch_size = 16
width = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [9]:
model = Sequential()
model.add(Convolution2D(width, (3, 3), padding="same", activation="relu", input_shape=input_shape))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(width*2, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*2, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(width*4, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*4, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(width*8, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*8, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(width*16, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*16, (3, 3), padding="same", activation="relu"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

opt1 = Adam(lr=1e-5)
opt2 = 'rmsprop'

model.compile(loss='binary_crossentropy',
              optimizer=opt2,
              metrics=['accuracy'])


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1921 images belonging to 2 classes.
Found 531 images belonging to 2 classes.


In [10]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('/Volumes/solo/ali/Data/model/my_model.h5')

Epoch 1/10
112/112 [==============================] - 114s - loss: 8.0068 - acc: 0.4978 - val_loss: 8.0676 - val_acc: 0.4940
Epoch 2/10
112/112 [==============================] - 111s - loss: 8.1752 - acc: 0.4872 - val_loss: 7.8227 - val_acc: 0.5093
Epoch 3/10
112/112 [==============================] - 112s - loss: 7.7405 - acc: 0.5145 - val_loss: 8.3508 - val_acc: 0.4762
Epoch 4/10
112/112 [==============================] - 111s - loss: 8.0240 - acc: 0.4967 - val_loss: 8.3838 - val_acc: 0.4741
Epoch 5/10
112/112 [==============================] - 110s - loss: 8.2464 - acc: 0.4827 - val_loss: 8.0537 - val_acc: 0.4948
Epoch 6/10
112/112 [==============================] - 111s - loss: 8.0952 - acc: 0.4922 - val_loss: 8.1319 - val_acc: 0.4899
Epoch 7/10
112/112 [==============================] - 111s - loss: 8.0596 - acc: 0.4945 - val_loss: 8.2518 - val_acc: 0.4824
Epoch 8/10
112/112 [==============================] - 111s - loss: 8.0774 - acc: 0.4933 - val_loss: 8.4498 - val_acc: 0.4700


# fenlei 

In [ ]:
def load_train():
    data_path = src
    folders = [x for x in os.listdir(data_path) if 'subset' in x]
    os.chdir(data_path)
    patients = []
    for i in folders:
        os.chdir(data_path + i)
        #print('Changing folder to: {}'.format(data_path + i))
        patient_ids = [x for x in os.listdir(data_path + i) if '.mhd' in x]
        for id in patient_ids:
            j = '{}/{}'.format(i, id)
            patients.append(j)
    return patients

def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)
        
def plot_ct_scan(scan):
    f, plots = plt.subplots(int(scan.shape[0] / 20) + 1, 4, figsize=(25, 25))
    for i in range(0, scan.shape[0], 5):
        plots[int(i / 20), int((i % 20) / 5)].axis('off')
        plots[int(i / 20), int((i % 20) / 5)].imshow(scan[i], cmap=plt.cm.bone) 
         
def print_mask(lung_m, nodule_m):
    fig, ax = plt.subplots(1,2, figsize=(20,16))
    ax[0].imshow(lung_m, cmap = plt.cm.bone)
    ax[1].imshow(nodule_m, cmap = plt.cm.bone)
    return

def seq(start, stop, step=1):
    n = int(round((stop - start)/float(step)))
    if n > 1:
        return([start + step*i for i in range(n+1)])
    else:
        return([])
    
def load_itk(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates

def get_pixels_hu(image):
    image = image.astype(np.int16)
    image[image == threshold_min] = 0
    return np.array(image, dtype=np.int16)

def get_nodule_slices(lung_mask, nodule_mask, lung_raw):
    indexes = np.unique(np.nonzero(nodule_mask)[0])
    #print('Nodule_present on slices: {}'.format(indexes))
    lung_mask_pres = lung_mask[indexes, :, :]
    nod_mask_pres = nodule_mask[indexes, :, :]
    lung_raw_pres = lung_raw[indexes, :, :]    
    return lung_mask_pres, nod_mask_pres, lung_raw_pres

def reshape_3d(image_3d):
    reshaped_img = image_3d.reshape([image_3d.shape[0], 1, 512, 512])
    #print('Reshaped image shape:', reshaped_img.shape)
    return reshaped_img




def create_masks_for_patient_watershed(img_file, save = True):
    def draw_nodule_mask(node_idx, cur_row):
        #print('Working on node: {}, row: {}'.format(node_idx, cur_row), '\n')
        coord_x = cur_row["coordX"]
        coord_y = cur_row["coordY"]
        coord_z = cur_row["coordZ"]
        diam = cur_row["diameter_mm"]
        radius = np.ceil(diam/2)
        noduleRange = seq(-radius, radius, RESIZE_SPACING[0])
        #print('Nodule range:', noduleRange)
        world_center = np.array((coord_z,coord_y,coord_x))   # nodule center
        voxel_center = world_2_voxel(world_center, origin, new_spacing)
        image_mask = np.zeros(lung_img.shape)
        for x in noduleRange:
            for y in noduleRange:
                for z in noduleRange:
                    coords = world_2_voxel(np.array((coord_z+z,coord_y+y,coord_x+x)),origin,new_spacing)
                    if (np.linalg.norm(voxel_center - coords) * RESIZE_SPACING[0]) < radius:
                        image_mask[int(np.round(coords[0])),int(np.round(coords[1])),int(np.round(coords[2]))] = int(1)
        #print(np.max(image_mask))    
      
        return image_mask

    print("Getting mask for image file {}".format(img_file))
    patient_id = img_file.split('/')[-1][:-4]
    mini_df = df_node[df_node["file"] == img_file]
    if mini_df.shape[0] > 0: # some files may not have a nodule--skipping those 
        img, origin, spacing = load_itk(src + img_file)
        height, width = img.shape[1], img.shape[2]
        #calculate resize factor
        RESIZE_SPACING = [1, 1, 1]
        resize_factor = spacing / RESIZE_SPACING
        new_real_shape = img.shape * resize_factor
        new_shape = np.round(new_real_shape)
        real_resize = new_shape / img.shape
        new_spacing = spacing / real_resize
        
        
        lung_img = scipy.ndimage.interpolation.zoom(img, real_resize)
        
        
        
        
        #print('Original image shape: {}'.format(img.shape))
        #print('Resized image shape: {}'.format(lung_img.shape))

        lung_img = get_pixels_hu(lung_img)
        #lung_mask = segment_lung_from_ct_scan(lung_img)
        #lung_mask[lung_mask >= threshold_max] = threshold_max
        #lung_img[lung_img >= threshold_max] = threshold_max
        #lung_img[lung_img == 0] = threshold_min
        
        
        
        lung_mask = lung_img.copy()
        #lung_mask[lung_mask == 0] = threshold_min
        lung_mask[lung_mask >= threshold_max] = threshold_max
        lung_img[lung_img >= threshold_max] = threshold_max
        
        lung_masks_512 = np.zeros([lung_img.shape[0], height_mask, width_mask], dtype = np.float32)
        nodule_masks_512 = np.zeros([lung_img.shape[0], height_mask, width_mask], dtype = np.float32)
        lung_masks_512[lung_masks_512 == 0] = threshold_min
        
        i = 0
        for node_idx, cur_row in mini_df.iterrows(): 
            nodule_mask = draw_nodule_mask(node_idx, cur_row)
            lung_img_512, lung_mask_512, nodule_mask_512 = np.zeros((lung_img.shape[0], 512, 512)), np.zeros((lung_mask.shape[0], 512, 512)), np.zeros((nodule_mask.shape[0], 512, 512))
            lung_mask_512[lung_mask_512 == 0] = threshold_min
            lung_img_512[lung_img_512 == 0] = threshold_min
            original_shape = lung_img.shape
            
            for z in range(lung_img.shape[0]):
                
                offset = (512 - original_shape[1])
                upper_offset = int(np.round(offset/2))
                lower_offset = int(offset - upper_offset)

                new_origin = voxel_2_world([-upper_offset,-lower_offset,0],origin,new_spacing)
                
                lung_img_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = lung_img[z,:,:]
                lung_mask_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = lung_mask[z,:,:]
                nodule_mask_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = nodule_mask[z,:,:]
            nodule_masks_512 += nodule_mask_512

        #print('Offsets shape for node index {} - main: {}, upper: {}, lower: {}'.format(node_idx, offset, upper_offset, lower_offset), '\n')
      
       
        lung_mask_pres, nod_mask_pres, lung_raw_pres = get_nodule_slices(lung_mask_512, nodule_masks_512, lung_img_512)
        #print('Nodules present on slices: ', np.unique(np.nonzero(nodule_masks_512)[0]))
        
        
        
        del lung_mask_512, nodule_masks_512, lung_img_512
        gc.collect()
        
        lung_mask_pres = reshape_3d(lung_mask_pres)
        nod_mask_pres = reshape_3d(nod_mask_pres)
        lung_mask_pres[lung_mask_pres <= threshold_min] = threshold_min
        lung_mask_pres[lung_mask_pres >= threshold_max] = threshold_max
        
        lung_mask_preproc = my_PreProc(lung_mask_pres)
        lung_mask_preproc = lung_mask_preproc.astype(np.float32)
        nod_mask_pres = (nod_mask_pres > 0.0).astype(np.float32)
        nod_mask_pres[nod_mask_pres == 1.0] = 255.

        np.save('{}/lung_mask/{}.npy'.format(dst_nodules, patient_id), lung_mask_preproc)
        np.save('{}/nodule_mask/{}.npy'.format(dst_nodules, patient_id), nod_mask_pres)
        
        del lung_mask_pres, lung_mask_preproc, nod_mask_pres
        gc.collect()
            
        #return lung_mask_preproc, nod_mask_pres
        return
    else: 
        print('\n', 'No nodules found for patient: {}'.format(patient_id), '\n')
        return
    
def predict_on_scan(scan):
    model = unet_model()
    model.load_weights(luna_path + 'Data/model/Unet_fulldatagen.h5')

    num_test = scan.shape[0]
    scan = scan.reshape(num_test, 1, 512, 512)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([scan[i:i+1]], verbose=0)[0]
    return imgs_mask_test

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
luna_path = '/Volumes/solo/ali/'
npy_path = luna_path + 'Data/test_mask/pre_lung_mask/'
nodemask = luna_path + 'Data/test_mask/pre_nodule_mask/'

pics_path = luna_path + 'pic/test/'

In [ ]:
file_list = []
subset_path = os.listdir(npy_path)
for _ in range(len(subset_path)):
    if subset_path[_] != '.DS_Store':
        file_list.append(npy_path + subset_path[_])

In [ ]:
df_npy = pd.read_csv('/Volumes/solo/ali/csv/test/annotations_pre.csv',index_col=None)
df_npy["npy_file"] = df_npy["seriesuid"].map(lambda file_name: get_filename(file_list, file_name))

In [ ]:
prob_ = []
for node_idx, row in tqdm(df_npy.iterrows()):
    img = np.load(row['npy_file'])
    img = np.squeeze(img)
    x = int(row['coordX'])
    y = int(row['coordY'])
    z = int(row['coordZ'])
    d = int(row['diameter_mm'])
    
    mask = np.zeros([32,32])
    rr, cc=draw.circle(y,x,d)
    draw.set_color(mask,[rr,cc],[1,])
    
    
    
    scipy.misc.imsave(pics_path + str(node_idx) + '.jpg', img[x,y-16:y+16,z-16:z+16])
    img1 = load_img(pics_path + str(node_idx) + '.jpg')
    x = img_to_array(img1).transpose(1,2,0)
    
    x = x.reshape((1,) + x.shape)
    cc = model.predict(x)
    prob_.append(cc[0][0])

In [ ]:
bb = pd.read_csv('/Volumes/solo/ali/csv/test/annotations.csv',index_col=None)
final_result = pd.DataFrame({'seriesuid':bb['seriesuid'].values,
             'coordX':bb['coordX'].values,
             'coordY':bb['coordY'].values,
             'coordZ':bb['coordZ'].values,
             'probability':prob_,
                            })
final_result = final_result[['seriesuid','coordX','coordY','coordZ','probability']]
final_result.to_csv('/Volumes/solo/ali/csv/test/annotations_final.csv',index=None)

In [ ]:
final_result

# 测试

In [15]:
file_list = []
subset_path = os.listdir('/Volumes/solo/ali/pic/train/0/')
for _ in range(len(subset_path)):
    if subset_path[_] != '.DS_Store':
        file_list.append('/Volumes/solo/ali/pic/train/0/' + subset_path[_])

In [16]:
#model.load_weights('/Volumes/solo/ali/Data/model/prob.h5')
#model.load('/Volumes/solo/ali/Data/model/my_model.h5')
#au = load_model('/Volumes/solo/ali/Data/model/my_model.h5')
prob_ = []
for _ in tqdm(file_list):

    img = load_img(_, target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)

    preds = model.predict(x)
    # decode the results into a list of tuples (class, description, probability)
    # (one such list for each sample in the batch)
    print('Predicted:', preds)

  2%|▏         | 16/964 [00:00<00:12, 74.96it/s]

('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))


  3%|▎         | 25/964 [00:00<00:12, 76.86it/s]

('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))


  4%|▍         | 43/964 [00:00<00:11, 78.85it/s]

...

('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))


 99%|█████████▉| 952/964 [00:11<00:00, 82.85it/s]

('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))


100%|██████████| 964/964 [00:11<00:00, 82.65it/s]

('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))
('Predicted:', array([[ 1.]], dtype=float32))


# 预训练模型

In [ ]:
img = load_img(_, target_size=(224, 224))

In [ ]:
from keras.applications.inception_v3 import InceptionV3
model1 = InceptionV3(include_top=True, weights=None,classes=1)


In [ ]:
model1.compile(loss='binary_crossentropy',
              optimizer=opt1,
              metrics=['accuracy'])


model1.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model1.save('/Volumes/solo/ali/Data/model/my_model.h5')